### Basic imports and setting up OpenAI and Tavily integration


In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=os.environ['OPENAI_API_KEY_TEG'],
    temperature=0,
)


### Moj własny tool


In [2]:
from langchain.tools import Tool

def calculate_loss(time_lost_hours, avg_salary_per_hour):
    return time_lost_hours * avg_salary_per_hour

agent2_tools = [
    Tool.from_function(
        func=calculate_loss,
        name="CalculateTimeLoss",
        description="Calculates company loss due to wasted time on social media. Inputs: time in hours, average hourly salary."
    )
]


### Definicja Tools

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.tools.tavily_search import TavilySearchResults
import os
from dotenv import load_dotenv
load_dotenv()

db_path = r"C:\PJATK\SEMESTR2\teg projekt\projekt\Logi-projektTEG\parser\logs.db"

sql_db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
sql_toolkit = SQLDatabaseToolkit(db=sql_db, llm=llm)  
agent1_tools = sql_toolkit.get_tools()
tavily = TavilySearchResults(max_results=5)


tools = agent1_tools + [tavily] + agent2_tools

### Defining the prompts for each agent role

In [15]:
search_template = """
                  Your job is to translate the following user question into an accurate SQL query based on the given table schema.
                  Identify the most relevant columns for answering the query.
                  Execute the SQL query and return the result.
                  Not permit risky SQL generation (e.g., DROP, DELETE) 
                  """
value_template = """Your job is to Calculate the financial loss caused by employees spending time on social media.

                    Return the estimated total cost of productivity loss.
                    Multiply total time lost (in hours) by the average monthly IT salary.
                    Your feedback should be in bullet point format only.
                    """
exchange_template = """Convert the total company losses into multiple currencies and assets: USD, EUR, GBP, BTC, XAU (gold).
                        Use the latest exchange rates for accurate conversion.
                        Return the result in a tabular format.
                    """

### Constructing the graph

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define method for creating agents and binding them to the tools
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    
    if tools:
      return prompt | llm.bind_tools(tools)
    else:
      return prompt | llm
    
search_agent = create_agent(
    llm,
    tools,
    system_message=search_template,
)

value_agent = create_agent(
    llm,
    tools,
    system_message=value_template,
)

exchange_agent = create_agent(
    llm,
    tools,
    system_message=value_template,
)

In [18]:
import functools

def agent_node(state, agent, name):
  result = agent.invoke(state)
  return {
      'messages': [result]
  }

search_node = functools.partial(agent_node, agent=search_agent, name="Search Agent")
value_node = functools.partial(agent_node, agent=value_agent, name="Value Agent")
exchange_node = functools.partial(agent_node, agent=exchange_agent, name="Exchange Agent")



In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Literal
from langchain_core.messages import AIMessage

MAX_ITERATIONS = 3  # na wszelki wypadek, choć niepotrzebne w tym konkretnym grafie

# 1. Definiujemy strukturę stanu
class AgentState(TypedDict):
    messages: List[BaseMessage]
    is_time_response: bool

# Funkcja decyzyjna
def check_if_time_response(state: AgentState) -> Literal["time_agent", "other_agent"]:
    content = state["messages"][-1].content.lower()
    if "time" in content or "response time" in content:
        return "time_agent"
    return "other_agent"

# Aktualizacja stanu
def update_state_with_time_flag(state: AgentState) -> AgentState:
    content = state["messages"][-1].content.lower()
    state["is_time_response"] = "time" in content or "response time" in content
    return state

# 3. Funkcja końcowa: odpowiedź językiem naturalnym
def natural_language_answer(state: AgentState) -> AgentState:
    return {
        "messages": state["messages"] + [AIMessage(content="Answer in natural language to the prompt")],
        "answer_in_time": False
    }


In [ ]:

from langgraph.graph import StateGraph, END

# Definicja węzłów (node functions)
def update_state_node(state: AgentState) -> AgentState:
    return update_state_with_time_flag(state)

# Węzły agentów (już zdefiniowane wcześniej)
# search_node, value_node, exchange_node

# Węzeł końcowy: naturalna odpowiedź
def final_answer_node(state: AgentState) -> AgentState:
    return natural_language_answer(state)

# Tworzymy graf
graph = StateGraph()

# Dodajemy węzły
graph.add_node("search_node", search_node)
graph.add_node("update_state", update_state_node)
graph.add_node("check_time", check_if_time_response)
graph.add_node("value_node", value_node)
graph.add_node("exchange_node", exchange_node)
graph.add_node("final_answer", final_answer_node)
graph.add_node(END)

# Krawędzie
graph.add_edge("search_node", "update_state")
graph.add_edge("update_state", "check_time")
graph.add_edge("check_time", "value_node", condition="time_agent")
graph.add_edge("check_time", "exchange_node", condition="other_agent")
graph.add_edge("value_node", "final_answer")
graph.add_edge("exchange_node", "final_answer")
graph.add_edge("final_answer", END)

# Stan początkowy
initial_state = {"messages": [], "is_time_response": False}

# Uruchomienie grafu
final_state = graph.run(initial_state)

# Wyświetlanie wyniku
print(final_state)

TypeError: StateGraph.__init__() got an unexpected keyword argument 'max_iterations'